In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="03.OutputParser")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 03.OutputParser]


### JsonOutputParser

사용자가 원하는 Json 스키마를 지정할 수 있게 해주는 도구입니다. 출력형태가 Json 입니다.  
사용자가 원하는 Json의 형태를 출력하려면 모델의 용량(인텔리전스)이 충분히 커야 합니다. 예를 들어, 3B, 8B, 30B, 70B 등등 파라미터의 갯수를 이야기합니다.  
용량이 크면 보다 복잡한 데이터를 처리하는데 유리합니다. 

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

원하는 출력의 구조, 스키마를 정의합니다.

In [6]:
# Pydatic 형식으로 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

`JsonOutputParser` 를 사용해서 파서를 구성합니다.

In [8]:
question = "비트코인에 대해 알려주세요."

# 파서를 설정
ouput_parser = JsonOutputParser(pydantic_object=Topic)
print(ouput_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


프롬프트를 설정합니다.

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시드던트입니다. 질문에 간결하게 답변해주세요."),
        ("human", "#Format:{format_instructions}\n\n#Question:{question}"),
    ]
)

prompt = prompt.partial(format_instructions=ouput_parser.get_format_instructions())

chain = prompt | llm | ouput_parser

response = chain.invoke({"question": question})

In [10]:
response

{'description': '비트코인은 분산형 디지털 통화로, 중앙은행이나 단일 관리 기관 없이 운영됩니다.',
 'hashtags': '#비트코인 #암호화폐'}

# Pydantic 없이 사용하는 방법, JsonOutputParser 템플릿(나중에 복사해서 사용하기)

In [11]:
# 질의 작성
question = "비트코인에 대해 알려주세요."

# 파서 설정
ouput_parser = JsonOutputParser()

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시드던트입니다. 질문에 간결하게 답변해주세요."),
        ("human", "#Format:{format_instructions}\n\n#Question:{question}"),
    ]
)

prompt = prompt.partial(format_instructions=ouput_parser.get_format_instructions())

chain = prompt | llm | ouput_parser

response = chain.invoke({"question": question})

print(response)

{'name': '비트코인', 'symbol': 'BTC', 'type': '암호화폐', 'created': '2009년', 'creator': '사토시 나카모토', 'technology': {'blockchain': True, 'decentralized': True, 'peer_to_peer': True}, 'features': ['탈중앙화', '투명성', '보안성', '한정된 공급'], 'use_cases': ['온라인 결제', '투자', '자산 보호'], 'market': {'volatility': '높음', 'adoption': '전 세계적으로 증가'}}
